# Vektorimuotoinen paikkatieto

Vektoriaineistot ovat paikkatietoa joko piste- viiva- tai polygon-muodossa. Tässä osiossa tutustutaan vektoridatan käsittelyn perusteisiin Pythonissa.

# Shapely ja geometriat

**Shapely** on vektoriaineistojen kannalta olennainen kirjasto, sillä se toteuttaa tuen geometrioiden luontiin, esittämiseen ja moniin spatiaalisiin operaatioihin. Shapelyä ei usein käytetä suoraan, mutta koska GeoPandasin geometriat perustuvat täysin Shapelyyn, on hyvä ymmärtää perusteet.

Aloitetaan ottamalla käyttöön Shapely-kirjastosta pisteet, viivat ja monikulmiot.

In [ ]:
from shapely import Point, LineString, Polygon

Voimme nyt luoda pisteen, eli **Point**-olion:

In [ ]:
point = Point(1, 2)
point

In [ ]:
print(point)

Voimme ottaa mukaan myös kolmannen ulottuvuuden:

In [ ]:
point_3d = Point(2, 4, 1)
point_3d

Tehdään seuraavaksi monta pistettä, ja muodostetaan niistä viiva (**LineString**-olio):

In [ ]:
point_1 = Point(0, 0)
point_2 = Point(-1, 1)
point_3 = Point(0.5, 1.5)
point_4 = Point(1, 0.5)

list_of_points = [point_1, point_2, point_3, point_4]

line = LineString(list_of_points)
line

Shapely-geometrioilla on monia hyödyllisiä attribuutteja. Tässä esimerkiksi viivan pituus ja massakeskipiste.

In [ ]:
print(line.length)
print(line.centroid)

Monikulmion luonti onnistuu samaan tapaan:

In [ ]:
polygon = Polygon(list_of_points)
polygon

Monikulmiolle voidaan laskea myös alue:

In [ ]:
print(polygon.centroid)
print(polygon.area)
polygon.buffer(1)

In [ ]:
from shapely import MultiPoint, MultiLineString, MultiPolygon

multipoint = MultiPoint(
    [Point(1, 2), Point(3, 4)]
)
multiline = MultiLineString(
    [LineString([(1, 7), (6, 4)]), LineString([(0, -1), (-4, 5)])]
)
multipolygon = MultiPolygon(
    [Polygon([(0, 0), (-10, 0), (2, 20)]), Polygon([(3, 6), (10, 6), (11, 20)])]
)
multipolygon

## Harjoitus - ympyrä

Tee Shapely-geometrioita ja metodeja hyödyntäen "ympyrä".
1. Vinkki: et tarvitse yhtäkään uutta `import`-komentoa.
2. Vinkki: tämä on hieman kompakysymys. Helpompaa voi olla ajatella: miten teen mahdollisimman pyöreän monikulmion?

In [ ]:
# Kirjoita ratkaisu


## Ratkaisu

In [ ]:
my_circle = Point(1,1).buffer(10)

print(type(my_circle))
my_circle

# GeoPandas

Seuraavaksi siirrytään vektoriaineistojen käsittelyssä erittäin monikäyttöiseen **GeoPandas**-kirjastoon.

In [ ]:
from pathlib import Path
import geopandas as gpd

In [ ]:
file_path = Path("./data/kunnat.gpkg")

municipalities = gpd.read_file(file_path)

In [ ]:
municipalities.head()

In [ ]:
type(municipalities)

In [ ]:
type(municipalities.geometry)

In [ ]:
type(municipalities["geometry"])

In [ ]:
type(municipalities.geometry.iloc[0])

In [ ]:
municipalities.plot()

In [ ]:
parainen = municipalities.loc[municipalities["nimi"] == "Parainen"]
parainen

In [ ]:
parainen.plot()

Kokeile myös DataFramen `explore`-metodia.

In [ ]:
parainen.explore()

In [ ]:
municipalities.area

In [ ]:
parainen.area

In [ ]:
municipalities.crs

In [ ]:
municipalities_wgs = municipalities.to_crs(epsg=4326)
municipalities_wgs.crs

In [ ]:
municipalities_wgs.plot(aspect="equal")

# 1. Taulukosta kartalle

In [ ]:
import pandas as pd

lightnings = pd.read_csv("./data/lightnings.csv")
lightnings.head()

In [ ]:
geo_lightnings = gpd.GeoDataFrame(lightnings)
geo_lightnings.head()

In [ ]:
geo_lightnings = geo_lightnings.set_geometry(
    gpd.points_from_xy(x=lightnings["longitude"], y=lightnings["latitude"])
)

geo_lightnings.head()

In [ ]:
geo_lightnings.plot(markersize=0.01)

In [ ]:
geo_lightnings.crs

In [ ]:
geo_lightnings = geo_lightnings.set_crs(epsg=4326)
geo_lightnings.crs

In [ ]:
geo_lightnings = geo_lightnings.to_crs(epsg=3067)
geo_lightnings.head()

# 2. Salamahavainnot kunnittain

In [ ]:
municipalities_with_lightnings = municipalities.sjoin(
    geo_lightnings,
    predicate="contains",
)

In [ ]:
municipalities_with_lightnings

In [ ]:
# Group by district name
group_by = municipalities_with_lightnings.groupby("nimi")

lightnings_by_municipality = group_by.size()
lightnings_by_municipality
#lightnings_by_municipality = lightnings_by_municipality.to_frame()
lightnings_by_municipality = pd.DataFrame(lightnings_by_municipality, columns=["lightning_count"])

In [ ]:
type(lightnings_by_municipality)

In [ ]:
municipalities = municipalities.merge(
    lightnings_by_municipality,
    left_on="nimi",
    right_on="nimi",
    how="left",
)

In [ ]:
municipalities.plot(
    column="lightning_count"
)

In [ ]:
area_km2 = municipalities.area / 1000000
area_km2

In [ ]:
municipalities["lightnings_per_km2"] = municipalities["lightning_count"] / area_km2
municipalities.head()

In [ ]:
municipalities.plot(column="lightnings_per_km2")

# 3. Tulosten visualisointi

In [ ]:
municipalities.plot(
    column="lightnings_per_km2",
    legend=True,
    figsize=(7, 7),
    cmap="Oranges",
)

In [ ]:
municipalities.plot(
    column="lightnings_per_km2",
    legend=True,
    figsize=(7, 7),
    scheme="quantiles",
)

In [ ]:
municipalities.plot(
    column="lightnings_per_km2",
    legend=True,
    figsize=(7, 7),
    scheme="naturalbreaks",
)

In [ ]:
m = municipalities.explore(
    column="lightnings_per_km2",
    tiles="CartoDB Positron",
    cmap="cividis",
    scheme="naturalbreaks",
    style_kwds={"fillOpacity": 1},
)

In [ ]:
m

In [ ]:
out_file_path = Path("./map.html")

m.save(out_file_path)